In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta

# CREATE TABLES
---

## create table 1 from mouse data base

In [2]:
table1 = pd.read_csv(r'.\data\table1.csv')
table1.rename(columns={'#':'Mouse ID'},inplace=True)
table1.head(1)

,Mouse ID,Genotype,Gender,Line,TgSWDI,Genotype 2,Genotype 3,Father,Mother,DOB,...,Birth status,Estimated Delivery Date,Delivery Date,Ween,Male Status,Breeding Status,Maze Configuration,Treatment Group,Virus Implantation,Experimental Status
0,170,Tg-SwDI,M,Tg-SwDI,Pos,NaN,NaN,Tg-SwDI (116),Tg-SwDI (117),8/2/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,ABFG,--,--,Testing


In [3]:
table1.to_csv(r'.\data\table1.csv',index=False)

# create table 2 from history data

In [4]:
data = pd.read_excel(r'.\data\RAM_Experimental_Data_2020_03_14_19_01_03.xlsx',sheet_name=None,index_col=False)
m_IDs = list(data.keys())
attributes = ['Date','Weight','Baited Arms Retrieved','Reference Memory Errors','Working Memory Errors','Other Notes']

data['ID_N171'][attributes].dtypes

Date                       object
Weight                      int64
Baited Arms Retrieved       int64
Reference Memory Errors     int64
Working Memory Errors       int64
Other Notes                object
dtype: object

In [5]:
import numpy as np

df = data['ID_N71'][attributes]
df['Mouse ID'] = np.empty_like(data['ID_N71']['Date'])

for i in range(0,len(data['ID_N71']['Date'])):
    df.loc[i,'Mouse ID'] = '71'

In [6]:
for m_ID in m_IDs:
    if m_ID != 'ID_N71':
        
        df_append = data[m_ID][attributes]

        for i in range(0,len(data[m_ID]['Date'])):
            df_append.loc[i,'Mouse ID'] = m_ID[4:]

        df = pd.concat([df,df_append])


dt = pd.to_datetime(df['Date'])
df['Date'] = dt

df.sort_values(by=['Date'],inplace=True)
df.reset_index(drop=True,inplace=True)
df = df[['Mouse ID','Date','Weight',
       'Baited Arms Retrieved','Reference Memory Errors','Working Memory Errors',
       'Other Notes']]

df.head(1)

,Mouse ID,Date,Weight,Baited Arms Retrieved,Reference Memory Errors,Working Memory Errors,Other Notes
0,85,2019-07-01,16,4,7,5,"Retrieved bait in arm A,C,F,D"


### Add time and summary columns to table2

In [7]:
g = os.walk(r'E:\Backup_Radial Arm Maze Experiments Folder') 

summ_list = []
avi_list = []

for path,dir_list,file_list in g:  
    for file_name in file_list: 
        if file_name[-5:] == "s.csv":
            summ_list.append(os.path.join(path, file_name))
        elif file_name[-4:] == ".avi":
            avi_list.append(os.path.join(path, file_name))

print(summ_list)
print(avi_list)

EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_16\\2020_02_16_N256_ABFG_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_16\\2020_02_16_N260_BDFH_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_16\\2020_02_16_N261_BDFH_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_16\\2020_02_16_N264_ABFG_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_22\\2020_02_22_EmptyBox1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_22\\2020_02_22_N131_ABFG_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_22\\2020_02_22_N177_ACDF_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_22\\2020_02_22_N178_ACDF_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_22\\2020_02_22_N181_BDFH_EB1_Trial1.avi', 'E:\\Backup_Radial Arm Maze Experiments Folder\\2020_02_22\\2020_02_22_N182_BDFH_EB1_Trial1.avi', 'E:\\Backup_Ra

In [8]:
time_columns = ['Start Time','End Time','Video Length']

summ_columns = ['File', 'FPS', 'Location_Thresh', 'Use_Window', 'Window_Weight', 'Window_Size', 
                'Start_Frame', 'bin', 'range(f)', 'Distance', 
                'Center', 'ArmA', 'ArmB', 'ArmC', 'ArmD', 'ArmE', 'ArmF', 'ArmG', 'ArmH']

columns = time_columns + summ_columns

for column in columns:
    df[column] = np.empty_like(df['Mouse ID'])

In [9]:
# find the corresponding record in table2 and append time data 

for avi_file in avi_list:
    try:
        avi_info = os.stat(avi_file)

        mtime = avi_info.st_mtime                    # This is the end time of the video
        mtime_str = datetime.datetime.fromtimestamp(mtime).strftime('%Y-%m-%d-%H-%M-%S')

        mtime_parse = mtime_str.split('-')

        mtime_stamp = pd.Timestamp(''.join(mtime_parse))
        summ_file = avi_file[:-4] + '_SummaryStats.csv'
        summ_info = pd.read_csv(summ_file)

        total_frames = eval(summ_info['range(f)'][0])[1]
        print('total_frames:',total_frames)

        seconds = round(total_frames/30) 
        print('seconds:',seconds)
        length = timedelta(weeks=0, days=0, hours=0,minutes=0, seconds=seconds)

        stime_stamp = mtime_stamp - length          # This is the start time of the video

        file_name = summ_info['File'][0].split('\\')[-1] # e.g.2020_01_16_N170_ABFG_EB1_Trial1.avi
        print('---------',file_name,'---------\n')

        date = '-'.join(file_name.split('_')[0:3])   # e.g.2020-01-16
        m_ID = file_name.split('_')[3][1:]           # e.g.170

        for row in range(0,df.shape[0]):
            if str(df.loc[row,'Date'])[0:10] == date and df.loc[row,'Mouse ID'] == m_ID:
                df.loc[row,'Start Time'] = stime_stamp
                df.loc[row,'End Time'] = mtime_stamp    
                df.loc[row,'Video Length'] = length
    except:
        pass

- 2020_02_15_N228_BDFH_EB1_Trial1.avi ---------

total_frames: 3481
seconds: 116
--------- 2020_02_15_N230_BDFH_EB1_Trial1.avi ---------

total_frames: 2019
seconds: 67
--------- 2020_02_15_N232_BDFH_EB1_Trial1.avi ---------

total_frames: 10335
seconds: 344
--------- 2020_02_15_N256_ABFG_EB1_Trial1.avi ---------

total_frames: 12599
seconds: 420
--------- 2020_02_15_N264_ABFG_EB1_Trial1.avi ---------

total_frames: 1930
seconds: 64
--------- 2020_02_15_N282_ACDF_EB1_Trial1.avi ---------

total_frames: 3235
seconds: 108
--------- 2020_02_15_N284_ACDF_EB1_Trial1.avi ---------

total_frames: 1063
seconds: 35
--------- 2020_02_15_N285_ACDF_EB1_Trial1.avi ---------

total_frames: 3478
seconds: 116
--------- 2020_02_15_N286_ACDF_EB1_Trial1.avi ---------

total_frames: 10127
seconds: 338
--------- 2020_02_15_N71_ABFG_EB1_Trial1.avi ---------

total_frames: 1133
seconds: 38
--------- 2020_02_15_N99_ABFG_EB1_Trial1.avi ---------

total_frames: 3267
seconds: 109
--------- 2020_02_16_N170_ABFG_E

In [10]:
# find the corresponding record in table2 and append summary data 
for summ_file in summ_list:
    
    summ_info = pd.read_csv(summ_file)
    print(summ_info['File'])
    try:
        file_name = summ_info['File'][0].split('\\')[-1] # e.g.2020_01_16_N170_ABFG_EB1_Trial1.avi
        date = '-'.join(file_name.split('_')[0:3])   # e.g.2020-01-16
        m_ID = file_name.split('_')[3][1:]           # e.g.170

        for row in range(0,df.shape[0]):

            if str(df.loc[row,'Date'])[0:10] == date and df.loc[row,'Mouse ID'] == m_ID:

                for summ_column in summ_columns:
                    df.loc[row,summ_column] = summ_info.loc[0,summ_column]     # write summary data to df
    except:
        pass

4_ACDF_EB2_Trial1.avi
Name: File, dtype: object
0    2020_02_07_N285_ACDF_EB2_Trial1.avi
Name: File, dtype: object
0    2020_02_07_N286_ACDF_EB2_Trial1.avi
Name: File, dtype: object
0    2020_02_07_N71_ABFG_EB2_Trial1.avi
Name: File, dtype: object
0    2020_02_07_N99_ABFG_EB2_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N131_ABFG_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N177_ACDF_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N178_ACDF_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N181_BDFH_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N182_BDFH_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N184_BDFH_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N190_BCEG_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N193_BCEG_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N194_BCEG_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_N196_BCEG_EB1_Trial1.avi
Name: File, dtype: object
0    2020_02_08_

In [11]:
df.to_csv(r'.\data\table2_update.csv',index=False)